# Feature squeeze implementation

In [1]:
import numpy as np
from scipy.stats import entropy
from scipy import ndimage

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

In [3]:
l1_dist = lambda x1,x2: np.sum(np.abs(x1 - x2), axis=tuple(range(len(x1.shape))[1:]))
l2_dist = lambda x1,x2: np.sum((x1-x2)**2, axis=tuple(range(len(x1.shape))[1:]))**.5

In [4]:
# Note: KL-divergence is not symentric.
# Designed for probability distribution (e.g. softmax output).
def kl(x1, x2):
    assert x1.shape == x2.shape
    # x1_2d, x2_2d = reshape_2d(x1), reshape_2d(x2)

    # Transpose to [?, #num_examples]
    x1_2d_t = x1.transpose()
    x2_2d_t = x2.transpose()

    # pdb.set_trace()
    e = entropy(x1_2d_t, x2_2d_t)
    e[np.where(e==np.inf)] = 2
    return e

In [5]:
def reduce_precision_py(x, npp):
    """
    Reduce the precision of image, the numpy version.
    :param x: a float tensor, which has been scaled to [0, 1].
    :param npp: number of possible values per pixel. E.g. it's 256 for 8-bit gray-scale image, and 2 for binarized image.
    :return: a tensor representing image(s) with lower precision.
    """
    # Note: 0 is a possible value too.
    npp_int = npp - 1
    x_int = np.rint(x * npp_int)
    x_float = x_int / npp_int
    return x_float

In [6]:
def median_filter_py(x, width, height=-1):
    """
    Median smoothing by Scipy.
    :param x: a tensor of image(s)
    :param width: the width of the sliding window (number of pixels)
    :param height: the height of the window. The same as width by default.
    :return: a modified tensor with the same shape as x.
    """
    if height == -1:
        height = width
    var = ndimage.filters.median_filter(x, size=(1,width,height,1), mode='reflect')
    #print("inside median filter")
    #print(type(var))
    return torch.from_numpy(var)

In [7]:
# Squeezers implemented in OpenCV
# OpenCV expects uint8 as image data type.
def opencv_wrapper(imgs, opencv_func, argv):
    ret_imgs = []
    imgs_copy = imgs

    if imgs.shape[3] == 1:
        imgs_copy = np.squeeze(imgs)

    for img in imgs_copy:
        img_uint8 = np.clip(np.rint(img * 255), 0, 255).astype(np.uint8)
        ret_img = opencv_func(*[img_uint8]+argv)
        if type(ret_img) == tuple:
            ret_img = ret_img[1]
        ret_img = ret_img.astype(np.float32) / 255.
        ret_imgs.append(ret_img)
    ret_imgs = np.stack(ret_imgs)

    if imgs.shape[3] == 1:
        ret_imgs = np.expand_dims(ret_imgs, axis=3)

    return 

In [8]:
def bit_depth_py(x, bits):
    precisions = 2**bits
    return reduce_precision_py(x, precisions)

In [9]:
def non_local_means_color_py(imgs, search_window, block_size, photo_render):
    import cv2
    ret_imgs = opencv_wrapper(imgs, cv2.fastNlMeansDenoisingColored, [None,photo_render,photo_render,block_size,search_window])
    return ret_imgs

In [10]:
m = nn.Softmax(dim=1)

In [11]:
def get_distance(model, dataset, X1):
    #X1_pred = model.predict(X1)
    X1_pred = m(model(X1))
    vals_squeezed = []

    if dataset == 'mnist':
        X1_seqeezed_bit = bit_depth_py(X1.cpu(), 1)
        #print(type(X1_seqeezed_bit))
        
        #model.predict is tf based. need torch based softmax. 
        #vals_squeezed.append(model.predict(X1_seqeezed_bit))
        vals_squeezed.append(m(model((X1_seqeezed_bit).to(device))))
        X1_seqeezed_filter_median = median_filter_py(X1.cpu(), 2)
        #print(("outside func",type(X1_seqeezed_filter_median)))
        vals_squeezed.append(m(model(X1_seqeezed_filter_median.to(device))))

    else:
        X1_seqeezed_bit = bit_depth_py(X1.cpu(), 5)
        #print(type(X1_seqeezed_bit))
        vals_squeezed.append(m(model((X1_seqeezed_bit).to(device))))
        X1_seqeezed_filter_median = median_filter_py(X1.cpu(), 2)
        #vals_squeezed.append(model(torch.from_numpy(X1_seqeezed_filter_median).float().to(device)))
        vals_squeezed.append(m(model((X1_seqeezed_filter_median).to(device))))
        #X1_seqeezed_filter_local = non_local_means_color_py(X1.cpu().numpy(), 13, 3, 2)
        #vals_squeezed.append(m(model((X1_seqeezed_filter_local).to(device))))

    dist_array = []
    for val_squeezed in vals_squeezed:
        dist = np.sum(np.abs(X1_pred.cpu().detach().numpy() - val_squeezed.cpu().detach().numpy()), axis=tuple(range(len(X1_pred.shape))[1:]))
        dist_array.append(dist)

    dist_array = np.array(dist_array)
    return np.max(dist_array, axis=0)

In [12]:
def train_fs(model, dataset, X1, train_fpr):
    distances = get_distance(model, dataset, X1)
    selected_distance_idx = int(np.ceil(len(X1) * (1-train_fpr)))
    threshold = sorted(distances)[selected_distance_idx-1]
    threshold = threshold
    #print ("Threshold value: %f" % threshold)
    return threshold

In [13]:
def get_distance_test(model, dataset, X1):
    #X1_pred = model.predict(X1)
    X1_pred = m(model(X1))
    vals_squeezed = []

    if dataset == 'mnist':
        X1_seqeezed_bit = bit_depth_py(X1.detach().cpu(), 1)
        #print(type(X1_seqeezed_bit))
        
        #model.predict is tf based. need torch based softmax. 
        #vals_squeezed.append(model.predict(X1_seqeezed_bit))
        vals_squeezed.append(m(model((X1_seqeezed_bit).to(device))))
        X1_seqeezed_filter_median = median_filter_py(X1.detach().cpu(), 2)
        #print(("outside func",type(X1_seqeezed_filter_median)))
        vals_squeezed.append(m(model(X1_seqeezed_filter_median.to(device))))

    else:
        X1_seqeezed_bit = bit_depth_py(X1.detach().cpu(), 5)
        #print(type(X1_seqeezed_bit))
        vals_squeezed.append(m(model((X1_seqeezed_bit).to(device))))
        X1_seqeezed_filter_median = median_filter_py(X1.detach().cpu(), 2)
        #vals_squeezed.append(model(torch.from_numpy(X1_seqeezed_filter_median).float().to(device)))
        vals_squeezed.append(m(model((X1_seqeezed_filter_median).to(device))))
        #X1_seqeezed_filter_local = non_local_means_color_py(X1.cpu().numpy(), 13, 3, 2)
        #vals_squeezed.append(m(model((X1_seqeezed_filter_local).to(device))))

    dist_array = []
    for val_squeezed in vals_squeezed:
        dist = np.sum(np.abs(X1_pred.cpu().detach().numpy() - val_squeezed.cpu().detach().numpy()), axis=tuple(range(len(X1_pred.shape))[1:]))
        dist_array.append(dist)

    dist_array = np.array(dist_array)
    return np.max(dist_array, axis=0)

In [14]:
def test(model, dataset, X, threshold):
    distances = get_distance_test(model, dataset, X)
    Y_pred = distances > threshold
    return Y_pred, distances

In [15]:
def compute_distance(model, dataset, X):
    distances = get_distance_test(model, dataset, X)
    return distances

In [16]:
def get_tpr_fpr(true_labels, pred_labels):
    TP = np.sum(np.logical_and(pred_labels == 1, true_labels == 1))
    FP = np.sum(np.logical_and(pred_labels == 1, true_labels == 0))

    AP = np.sum(true_labels)
    AN = np.sum(1-true_labels)

    tpr = TP/AP if AP>0 else np.nan
    fpr = FP/AN if AN>0 else np.nan

    return tpr, fpr, TP, AP, FP, AN

In [17]:
def evaluate_test(model, x_test, threshold, dataset):
    Y_all = np.concatenate([np.ones(len(x_test), dtype=bool)])
    #print(Y_all)
    Y_all_pred, Y_all_pred_score = test(model, dataset, x_test, threshold)
    tpr, fpr, tp, ap, fp, an = get_tpr_fpr(Y_all, Y_all_pred)
    return tpr
                                           

# MNIST thresholds

In [49]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [50]:
#for natural and adversarial LeNet Model 
class LeNet_normal(torch.nn.Module):
    """Network architecture from: https://github.com/ChawDoe/LeNet5-MNIST-PyTorch."""
    def __init__(self):
        super().__init__()
        self.conv_1 = torch.nn.Conv2d(1, 6, 5)
        self.pool_1 = torch.nn.MaxPool2d(2, 2)
        self.relu_1 = torch.nn.ReLU()
        self.conv_2 = torch.nn.Conv2d(6, 16, 5)
        self.pool_2 = torch.nn.MaxPool2d(2, 2)
        self.relu_2 = torch.nn.ReLU()
        self.fc_1 = torch.nn.Linear(256, 120)
        self.relu_3 = torch.nn.ReLU()
        self.fc_2 = torch.nn.Linear(120, 84)
        self.relu_4 = torch.nn.ReLU()
        self.fc_3 = torch.nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool_1(self.relu_1(self.conv_1(x)))
        x = self.pool_2(self.relu_2(self.conv_2(x)))
        x = x.view(x.shape[0], -1)
        x = self.relu_3(self.fc_1(x))
        x = self.relu_4(self.fc_2(x))
        x = self.fc_3(x)
        return x

In [51]:
def load_mnist_model(path):
    model = LeNet_normal()
    model.to(device)
    model.load_state_dict(torch.load(path))
    model.to('cuda')
    model.train(False)
    return model

In [52]:
path = "mnist_model.pth"
mnist_model = load_mnist_model(path)
mnist_model.to(device)
mnist_model.eval()

LeNet_normal(
  (conv_1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool_1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu_1): ReLU()
  (conv_2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool_2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu_2): ReLU()
  (fc_1): Linear(in_features=256, out_features=120, bias=True)
  (relu_3): ReLU()
  (fc_2): Linear(in_features=120, out_features=84, bias=True)
  (relu_4): ReLU()
  (fc_3): Linear(in_features=84, out_features=10, bias=True)
)

In [53]:
#get dataset
train_set = torchvision.datasets.MNIST(root='./sample_data', train=True, transform=torchvision.transforms.ToTensor(), download=True)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=10, pin_memory=True)

In [54]:
FPR = [0.01,0.05,0.1]
for fpr in FPR:
    t=[]
    for step, (x_batch, y_batch) in enumerate(train_loader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        threshold = train_fs(mnist_model, "mnist", x_batch, fpr)
        t.append(threshold)
        if step==100:
            break
    print("Threshold for {} FPR is {}.".format(fpr, sum(t)/len(t)))

/tmp/ipykernel_2163/987440447.py:11: DeprecationWarning: Please use `median_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  var = ndimage.filters.median_filter(x, size=(1,width,height,1), mode='reflect')


Threshold for 0.01 FPR is 0.5400584977487417.
Threshold for 0.05 FPR is 0.5400584977487417.
Threshold for 0.1 FPR is 0.059712059444638435.


In [55]:
from cleverhans.torch.attacks.fast_gradient_method import fast_gradient_method

def make_fgsm_attack(x_batch, y_batch, eps, normal_model): 
    
    images_pgd = fast_gradient_method(normal_model, x_batch, eps, np.inf)
    _, y_pred_pgd = normal_model(images_pgd).max(1)
    index = (y_pred_pgd != y_batch)
    pgd_images = images_pgd[index]
    y_pred_pgd = y_pred_pgd[index]
    return pgd_images, y_pred_pgd

In [73]:
# testing test function
threshold = 0.011
t=[]
for step, (x_batch, y_batch) in enumerate(train_loader):
    x_batch, y_batch = x_batch.to(device), y_batch.to(device)
    images_adv,y_pred_adv = make_fgsm_attack(x_batch, y_batch, 0.15, mnist_model)
    images_adv, y_pred_adv = images_adv.to(device), y_pred_adv.to(device)
    #tpr = evaluate_test(mnist_model, images_adv, threshold, "mnist")
    d = compute_distance(mnist_model, "mnist", images_adv)
    t.extend(d)
    if step==50:
        break
#avg = sum(t)/len(t)
print('TPR is ', avg)

/tmp/ipykernel_2163/987440447.py:11: DeprecationWarning: Please use `median_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  var = ndimage.filters.median_filter(x, size=(1,width,height,1), mode='reflect')


TPR is  1.6921674875925565


In [71]:
t

[0.0037023888,
 1.4547563,
 1.9398155,
 1.9642309,
 1.9949434,
 1.9999357,
 1.8329766,
 1.82579,
 1.9863436,
 1.6510091,
 1.8830819,
 1.7945919,
 1.6853534,
 1.999431,
 1.4717724,
 1.7163935,
 1.7187643,
 1.4445529,
 0.31661597,
 1.9820156,
 0.69838613,
 1.2439761,
 1.8433343,
 1.9856117,
 1.9242642,
 1.8001755,
 1.607698,
 0.8374935,
 0.5282521,
 1.9833698,
 1.7635174,
 0.7651484,
 1.975861,
 1.9880788,
 1.5262078,
 1.9976143,
 1.9947714,
 1.9979838,
 1.9735583,
 1.214601,
 1.9197868,
 1.9992412,
 1.9748027,
 1.4058942,
 1.9893807,
 1.7211668,
 1.9999855,
 1.9739358,
 0.14934598,
 1.9302452,
 0.9445748,
 1.3355427,
 1.6497753,
 1.9679126,
 1.9794323,
 1.8673459,
 1.9780377,
 1.950849,
 1.9072189,
 1.9990618,
 0.32116923,
 1.9619154,
 1.9953034,
 1.99959,
 1.99879,
 1.9403043,
 1.9644465,
 1.9987391,
 1.8372264,
 1.998923,
 1.904904,
 1.630156,
 1.8219489,
 1.9664894,
 0.62454545,
 1.9222841,
 1.9922737,
 1.9150343,
 1.3534813,
 1.8638101,
 1.9197011,
 1.9988408,
 1.9413468,
 1.9328399

In [64]:
import pandas as pd 
df = pd.DataFrame([d], index=["distance"])

In [65]:
df.to_csv('test.csv')

In [24]:
#from rev2.cifar10.model_utils import resnet50, CIFAR10_RESNET50_CKPT_PATH

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x, out_keys=None):
        out = {}
        x = self.conv1(x)
        out["c1"] = x
        x = self.bn1(x)
        out["bn1"] = x
        x = F.relu(x)
        out["r1"] = x

        x = self.layer1(x)
        out["l1"] = x
        x = self.layer2(x)
        out["l2"] = x
        x = self.layer3(x)
        out["l3"] = x
        x = self.layer4(x)
        out["l4"] = x

        x = F.avg_pool2d(x, 4)
        out["gvp"] = x
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        out["fc"] = x

        if out_keys is None:
            return x
        res = {}
        for key in out_keys:
            res[key] = out[key]
        return res


def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])


def ResNet34():
    return ResNet(BasicBlock, [3,4,6,3])


def resnet50():
    return ResNet(Bottleneck, [3,4,6,3])


def ResNet101():
    return ResNet(Bottleneck, [3,4,23,3])


def ResNet152():
    return ResNet(Bottleneck, [3,8,36,3])


def test():
    net = ResNet18()
    y = net(torch.randn(1,3,32,32))
    print(y.size())

# CIFAR thresholds

In [29]:
def load_cifar_model(path):
    model = resnet50()
    ckpt_dict = torch.load(path, lambda storage, loc: storage)
    model.load_state_dict(ckpt_dict)
    model.to('cuda')
    model.train(False)
    return model

In [30]:
path = "cifar.ckpt"
normal_model = load_cifar_model(path)
normal_model.to(device)
normal_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): Bottleneck(
      (

In [31]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                           download=True, transform=torchvision.transforms.ToTensor())
train_loader = DataLoader(trainset, shuffle=True, batch_size=10)

Files already downloaded and verified


In [32]:
FPR = [0.01,0.05,0.1]
for fpr in FPR:
    t=[]
    for step, (x_batch, y_batch) in enumerate(train_loader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        threshold = train_fs(normal_model, "cifar", x_batch, fpr)
        t.append(threshold)
        if step==100:
            break
    print("Threshold for {} FPR is {}.".format(fpr, sum(t)/len(t)))

/tmp/ipykernel_2163/987440447.py:11: DeprecationWarning: Please use `median_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  var = ndimage.filters.median_filter(x, size=(1,width,height,1), mode='reflect')


Threshold for 0.01 FPR is 1.155263510995889.
Threshold for 0.05 FPR is 1.143712807310474.
Threshold for 0.1 FPR is 0.3224780978692945.


# ImageNet threshold

In [18]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [19]:
def load_imagenet_model():
    model=torchvision.models.mobilenet_v3_small(weights=True).to(device)
    model.to('cuda')
    model.train(False)
    return model

In [20]:
normal_model = load_imagenet_model()
normal_model.to(device)
normal_model.eval()

/data/virtual environments/adv detection by robustness/adv_detection/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), 

In [22]:
# the validation transforms
valid_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])

In [23]:
#get dataset
images = '/home/db1702/Downloads/imagenet-mini/val/'
train = torchvision.datasets.ImageFolder(images, transform=valid_transform)
train_loader = DataLoader(train, shuffle=True, batch_size = 5)


In [25]:
FPR = [0.01,0.05,0.1]
for fpr in FPR:
    t=[]
    for step, (x_batch, y_batch) in enumerate(train_loader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        threshold = train_fs(normal_model, "cifar", x_batch, fpr)
        t.append(threshold)
        if step==200:
            break
    print("Threshold for {} FPR is {}.".format(fpr, sum(t)/len(t)))

/tmp/ipykernel_79836/987440447.py:11: DeprecationWarning: Please use `median_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  var = ndimage.filters.median_filter(x, size=(1,width,height,1), mode='reflect')


Threshold for 0.01 FPR is 1.3641814877144733.
Threshold for 0.05 FPR is 1.3909228329931325.
Threshold for 0.1 FPR is 1.2991581189988264.
